In [1]:
import h5py as h5
from h5glance import H5Glance
import numpy as np
import gvar as gv
# import pandas as pd
import os 
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib notebook
#import h5ls
import lsqfit
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import yaml
from pathlib import Path

In [2]:
#import local files
#sys.path.append("../")
# import data_loader as dl
# import fitter as fit
# import fit_manager as fm
import fitter.fit_collect as myfit

matplotlib.rcParams['figure.figsize'] = [10, 8]

In [3]:
os.getcwd()

'/home/gbradley/c51_corr_analysis/tests'

In [4]:
def read_dataset(inputfiles, grep=None, keys=None, h5group=None, binsize=1, tcol=0, Gcol=1,):
    if not hasattr(inputfiles, 'keys'):
        # inputfiles is a filename or list of filenames (or files)
        if h5group == [] or h5group is None:   # not needed after next gvar update
            h5group = '/'
        dset = gv.dataset.Dataset(
            inputfiles, binsize=binsize, grep=grep,
            h5group=h5group, keys=keys,
            )
    return dset 

In [5]:
directory = '../tests/data/C13/'
N_cnf = len([name for name in os.listdir(directory) if os.path.isfile(name)])
dirs = os.listdir( directory )
cnf_abbr = [files.split(".ama.h5",1)[0] for files in dirs]
datatag = []
for abbr in cnf_abbr:
    datatag.append(abbr.replace("-","."))
# print(datatag) 
data_file_list = list()
for dirpath,_,filenames in os.walk(directory):
    for f in filenames:
        data_file_list.append(os.path.abspath(os.path.join(dirpath, f)))
data_file_list[0]
twopt = []
threept = []
twopt.append(read_dataset(inputfiles=data_file_list[0], h5group='/2pt/proton/src5.0_snk5.0/proton/C13.b_5682/' ))
#     twopt.append(read_dataset(inputfiles=data_file_list[i], h5group='/2pt/proton/src5.0_snk5.0/proton/'+i+'/'))
print(twopt)

In [6]:
#ens = 
p_dict = {
    'abbr' : 'C13.b_5682', #CHANGE THIS
    'part' : 'proton', #CHANGE THIS # 'proton'
    'particles' : ['axial_num','vector_num','proton'],#['nucleon','axial_num', 'vector_num'],
    'sinks' : ['dir', 'smr'],

    't_range' : {
        'corr' : [7, 19], #change these
        'gA'   : [4, 12],
        'gV'   : [4, 12],
    },
    'n_states' : {
        'corr' : 2,
        'gA'   : 3,
        'gV'   : 3,},
         #for a15 ensemble, 4 gives less fit curvature 
    'pt2_nstates': 3, 
    'pt3_nstates': 3,
    'sum_nstates' : 3,

    'make_plots' : True,
    'save_prior' : False,
    
    'show_all' : True,
    'show_many_states' : False, # This doesn't quite work as expected: keep false
    'use_prior' : True
}

In [7]:
directory = '../tests/data/C13/'
N_cnf = len([name for name in os.listdir(directory) if os.path.isfile(name)])

dirs = os.listdir( directory )

cnf_abbr = [files.split(".ama.h5",0) for files in dirs]
data_file_list = list()
for dirpath,_,filenames in os.walk(directory):
    for f in filenames:
        data_file_list.append(os.path.abspath(os.path.join(dirpath, f)))
# print(data_file_list)
twopt_SS_paths = []
twopt_SS_paths.append('/2pt/proton/src5.0_snk5.0/proton/C13.b_5682/AMA')
twopt_PS_paths = []
twopt_PS_paths.append('/2pt/proton_SP/src5.0_snk5.0/proton/C13.b_5682/AMA')

threept_8_axial_paths = []
threept_8_axial_paths.append('/3pt_tsep8/NUCL_D_MIXED_NONREL_l0_g1/src5.0_snk5.0/qz+0_qy+0_qx+0/C13.b_5682/AMA')
threept_8_vector_paths = []
threept_8_vector_paths.append('/3pt_tsep8/NUCL_D_MIXED_NONREL_l0_g8/src5.0_snk5.0/qz+0_qy+0_qx+0/C13.b_5682/AMA')

data = {}
with h5.File(data_file_list[0],'r') as h5f:
    for i in range(len(threept_8_axial_paths)):
        for ii in range(len(threept_8_vector_paths)):
            for j in range(len(twopt_SS_paths)):
                for k in range(len(twopt_PS_paths)):
                    n_ga= np.array(h5f[threept_8_axial_paths[i]][:])
                    n_gv = np.array(h5f[threept_8_vector_paths[ii]][:])
                    n_ss = np.array(h5f[twopt_SS_paths[j]][:])
                    n_ps = np.array(h5f[twopt_PS_paths[k]][:])
                    data['3pt_tsep8_axial'] = n_ga.real
                    data['3pt_tsep8_vector'] = n_gv.real

                    data['2pt_SS'] = n_ss
                    data['2pt_PS'] = n_ps
                    # 
                    # print(data)


In [8]:
#load data into dictionaries to prepare for gvar averaging
#gvar averaging is done in fit_ensemble class within fit_manager

nucleon_corr_data = None
if 'proton' in p_dict['particles']:
    nucleon_corr_data = {}
    nucleon_corr_data['SS'] = data['2pt_SS'][:] #change
    nucleon_corr_data['PS'] = data['2pt_PS'][:]
    print(nucleon_corr_data)
        
axial_num_data = None
if 'axial_num' in p_dict['particles']:
    axial_num_data = {}
    axial_num_data['3pt_tsep8'] = data['3pt_tsep8_axial'][:] #change
    print(axial_num_data)

vector_num_data = None
if 'vector_num' in p_dict['particles']:
    vector_num_data = {}
    vector_num_data['3pt_tsep8'] = data['3pt_tsep8_vector']
    print(vector_num_data.keys())

{'SS': array([(9.09149800e-08, -6.58455149e-11),
       (2.98992592e-08, -1.43479335e-10),
       (1.32228739e-08,  1.94926986e-10),
       (6.35330028e-09,  9.39257271e-11),
       (3.17144105e-09,  1.95027154e-11),
       (1.65237520e-09,  2.92999464e-12),
       (8.86341924e-10, -4.06171492e-13),
       (4.69405783e-10,  4.13173084e-12),
       (2.48703047e-10,  9.99258012e-12),
       (1.37899315e-10,  8.22049542e-12),
       (7.41919662e-11,  9.22377420e-12),
       (3.75972273e-11,  2.11107440e-12),
       (1.92381198e-11, -1.00914399e-12),
       (9.67925279e-12,  4.58107397e-13),
       (5.32965921e-12,  1.60606786e-12),
       (3.65374976e-12,  1.32953131e-12),
       (1.29335445e-12,  5.09534069e-13),
       (1.80396302e-12, -4.41042654e-13),
       (3.59368308e-13, -7.57344046e-13),
       (8.36213159e-13, -8.30246621e-13),
       (5.42699783e-13, -5.28841539e-13),
       (3.53329585e-13, -1.77249189e-13),
       (3.73766897e-13, -2.18199369e-13),
       (3.02166010e-13, -4.

## pull in priors from csv in ens dirs, change csv as see fit ##

In [9]:
print(p_dict['pt2_nstates'])

3


In [10]:
fit = myfit.fitter(t_range=p_dict['t_range'], n_states=p_dict['n_states'], 
                    pt2_nstates=p_dict['pt2_nstates'], pt3_nstates= p_dict['pt3_nstates'],
                                sum_nstates=p_dict['sum_nstates'], 
                               nucleon_corr_gv=nucleon_corr_data, 
                               axial_num_gv=axial_num_data,
                               vector_num_gv=vector_num_data)
print(fit.get_fit())
if p_dict['save_prior']:
    dl.save_prior(fit_ensemble.make_prior_from_fit(), abbr=p_dict['abbr'])

TypeError: invalid type promotion

In [ ]:
fit_ensemble.make_prior_from_fit()

NameError: name 'fit_ensemble' is not defined

In [ ]:
#pickle gvar priors/posteriors from fit using gvar dump
fit_out = fit_ensemble.get_fit()
fit_dump = {}
fit_dump['prior'] = fit_out.prior
fit_dump['p'] = fit_out.p
fit_dump['logGBF'] = fit_out.logGBF
fit_dump['Q'] = fit_out.Q
print(fit_dump)

gv.dump(fit_dump,'ga_fit_results/{1}/{0}_{1}.p'.format(p_dict['part'],p_dict['abbr']))

((7, 19, 2), (4, 12, 3), (4, 12, 3))


IndexError: list index out of range

In [ ]:
#reassemble data from above 
#g = gv.load('ga_fit_results/{1}/{0}_{1}.p'.format(p_dict['part'],p_dict['abbr']))
#g

In [ ]:
#comment out plot commands in fit_manager for ga, gv for hp ens
if p_dict['make_plots']:
    plots = fit_ensemble.make_plots()
    output_dir = 'ga_fit_results/{0}/{1}_{0}.pdf'.format(p_dict['abbr'],p_dict['part'])
    output_pdf = PdfPages(output_dir)
    for plot in plots:
        if plot is not None:
            output_pdf.savefig(plot)
    if p_dict['show_many_states']:
        output_pdf.savefig(fit_ensemble.plot_stability(model_type='corr', n_states_array=[1, 2, 3, 4]))
        #output_pdf.savefig(fit_ensemble.plot_stability(model_type='gA', n_states_array=[1, 2, 3, 4]))
        #output_pdf.savefig(fit_ensemble.plot_stability(model_type='gV', n_states_array=[1, 2, 3, 4]))
    output_pdf.close()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## bootstrapping ##

bs resampling and redo fits 

keep track of seed 

gauge average bc approximate integral that way -> physical point

data from configs, resample gauge configs, average them to get result of integral

rand sample m times (gauge field configs) to compute bs sample 

array n x m, average over m (n number of resamples)

each resample n correspond to an integral to be computed by avg over m configurations

https://lsqfit.readthedocs.io/en/latest/lsqfit.html
1) make a large number of “bootstrap copies” of the original input data and prior that differ from each other by random amounts characteristic of the underlying randomness in the original data

2) repeat the entire fit analysis for each bootstrap copy of the data, extracting fit results from each 

3) use the variation of the fit results from bootstrap copy to bootstrap copy to determine an approximate probability distribution (possibly non-gaussian) for the fit parameters and/or functions of them: the results from each bootstrap fit are samples from that distribution.

In [ ]:
def bs_to_gvar(data, bs_list):
    bs_M = data['smr'].shape[0] # bs_M would be the same if you used 'ps' instead
    #bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
    
    temp_dict = {}
    for key in data.keys():
        temp = data[key][bs_list[0:]
        temp_dict[key] = np.mean(temp, axis=0).real
        
    for k in range(1, bs_N):
        for key in data.keys():
            temp = data[key][bs_list[k:]
            temp_dict[key] = np.vstack((temp_dict[key], np.mean(temp, axis=0).real))
    
    output = {}
    #return temp_dict
    for key in data.keys():
        mean = np.mean(data[key], axis=0)
        unc = np.cov(temp_dict[key], rowvar=False)
        output[key] = gv.gvar(mean,unc)
    return output

nucleon_bs_list = bs_to_gvar(nucleon_corr_data, 500)
axial_bs_list   = bs_to_gvar(axial_fh_num_data,500)
vector_bs_list  = bs_to_gvar(vector_fh_num_data,500)
#nucleon_bs_list


SyntaxError: invalid syntax (<ipython-input-515-3bdba660a7e5>, line 8)

In [ ]:
#b = gv.dataset.bootstrap_iter(nucleon_bs_list)
#print(next(b))b

In [ ]:
bs_list_path = os.path.realpath('./c51_mdwf_hisq_spec.h5')
#H5Glance(bs_list_path)

In [ ]:
with h5.File(bs_list_path, "r") as f: 
    bs = f["/"+ p_dict['abbr']+"/bs_lst"]
    bs_list = np.array(bs)
    print(bs_list)

OSError: Unable to open file (unable to open file: name = '/Users/grantdb/hyperon_ga/c51_mdwf_hisq_spec.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#bs_list[1,:]

In [ ]:
print(len(bs_list))
# bs_M = nucleon_corr_data['smr'].shape[0] # bs_M would be the same if you used 'ps' instead
bs_N = 1000 # In real-world cases, this would probably be much larger
# bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
# bs_list
#bs_list[:,:]

NameError: name 'bs_list' is not defined

In [ ]:
n_time_slices = nucleon_corr_data['smr'].shape[1] # again, could use 'ps' instead
bs_correlators_gv = []
for k in range(bs_N):
    temp = ({key : nucleon_corr_data[key][bs_list[k,:], :]
                           for key in nucleon_corr_data.keys()})
    
    # convert each bootstrap resample into a correlator with uncertainty
    # by using gv.dataset.avg_data (same function we used to make original data for fit)
    bs_correlators_gv.append(gv.dataset.avg_data(temp))
#bs_correlators_gv

In [ ]:
fit_ensemble_bs = fm.fit_ensemble(t_range=p_dict['t_range'],
                               n_states=p_dict['n_states'], prior=prior, 
                               nucleon_corr_data=bs_correlators_gv, 
                               axial_fh_num_data=axial_fh_num_data,
                               vector_fh_num_data=vector_fh_num_data)
print(fit_ensemble_bs)

In [ ]:
# Make bs_N fits, one for each of the resampled correlators
fit_keys = fit_out.p.keys()
output = {key : [] for key in fit_keys}

for j, bsfit in enumerate(fit_out.bootstrapped_fit_iter(datalist=bs_correlators_gv)):
    for key in fit_keys:
        # Save the best estimate for the central value 
        # of each parameter of each fit
        p = bsfit.pmean[key]
        output[key].append(p)

# # print results -- should be similar to previous results
table = gv.dataset.avg_data(output, bstrap=True)
print(gv.tabulate(table))

In [ ]:
#print(gv.evalcorr(table[key][:2])[1,0])
# measure of how well multi-dimensional Gaussian distributions(regular fit and bootstrapped fit) agree with each other 
# do their means agree within errors for corresponding elements. 
# note they share the same keys, so just loop over those 
# only parts of fit params that overlap are used 
for key in fit_keys:
    chi2 = gv.chi2(table[key],fit_out.p[key])
gv.fmt_chi2(chi2)

In [ ]:
print(fit_ensemble.get_fit())